In [1]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine, text

In [2]:
load_dotenv()

True

In [3]:
json_data = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/").json()

In [4]:
df = pd.DataFrame(json_data["elements"])

In [5]:
df = df[["first_name", "second_name", "web_name","total_points", "transfers_in", "transfers_in_event", "transfers_out", "transfers_out_event", "region"]]

In [6]:
engine = create_engine(os.getenv("POSTGRES_URI") or "")


In [7]:
with engine.begin() as con:
    con.execute(
        text(
            '''
            create table if not exists stg_players(
                player_id int generated always as identity primary key not null,
                first_name varchar(100) not null,
                second_name varchar(100) not null,
                web_name varchar(100) not null,
                total_points int,
                transfers_in int,
                transfers_in_event int,
                transfers_out int,
                transfers_out_event int,
                region int
            )
            '''
        )
    )
    
    con.execute(
        text(
            "truncate table stg_players restart identity"
        )
    )

In [8]:
df.to_sql(
    name="stg_players",
    con=engine,
    if_exists="append",
    index=False
)

817

In [9]:
with engine.begin() as con:
    con.execute(
        text(
            '''
            MERGE INTO players AS target
            USING stg_players AS source
            ON target.player_id = source.player_id
            
            WHEN MATCHED THEN
                UPDATE SET 
                    first_name = source.first_name,
                    second_name = source.second_name,
                    web_name = source.web_name,
                    total_points = source.total_points,
                    transfers_in = source.transfers_in,
                    transfers_in_event = source.transfers_in_event,
                    transfers_out = source.transfers_out,
                    transfers_out_event = source.transfers_out_event,
                    region = source.region
            
            WHEN NOT MATCHED THEN
                INSERT (first_name, second_name, web_name, total_points, transfers_in, transfers_in_event, transfers_out, transfers_out_event, region)
                VALUES (source.first_name, source.second_name, source.web_name, source.total_points, source.transfers_in, source.transfers_in_event, source.transfers_out, source.transfers_out_event, source.region)
            '''
        )
    )